## First, install the python dependencies

In [1]:
!pip install --upgrade pip && pip install -r requirements.txt

In [11]:
!python run.py

Model was already trained in a previous run
Submitting a prediction to the model with data {'zipcode': [76104], 'dob_ssn': ['19630621_4278'], 'person_age': [133], 'person_income': [59000], 'person_home_ownership': ['RENT'], 'person_emp_length': [123.0], 'loan_intent': ['PERSONAL'], 'loan_amnt': [35000], 'loan_int_rate': [16.02]}
Data returned from online store for request:
         dob_ssn  total_debt_due  credit_card_due  mortgage_due  \
0  19630621_4278        473071.0             3343        378847   

   student_loan_due  vehicle_loan_due  loan_amnt  
0             44375             11506      35000  
Loan rejected!


## Explore some entities, etc. on Feast

In [23]:
!feast -c feature_repo data-sources list

NAME              CLASS
Credit history    <class 'feast.infra.offline_stores.file_source.FileSource'>
Zipcode source    <class 'feast.infra.offline_stores.file_source.FileSource'>
application_data  <class 'feast.data_source.RequestSource'>


In [18]:
!feast -c feature_repo entities list

NAME     DESCRIPTION                                                   TYPE
dob_ssn  Date of birth and last four digits of social security number  ValueType.STRING
zipcode                                                                ValueType.INT64


In [20]:
!feast -c feature_repo feature-views list

NAME              ENTITIES     TYPE
credit_history    {'dob_ssn'}  FeatureView
zipcode_features  {'zipcode'}  FeatureView
total_debt_calc   {'dob_ssn'}  OnDemandFeatureView


In [22]:
!feast -c feature_repo on-demand-feature-views describe total_debt_calc

spec:
  name: total_debt_calc
  features:
  - name: total_debt_due
    valueType: DOUBLE
  sources:
    application_data:
      requestDataSource:
        type: REQUEST_SOURCE
        requestDataOptions:
          schema:
          - name: loan_amnt
            valueType: INT64
        name: application_data
    credit_history:
      featureViewProjection:
        featureViewName: credit_history
        featureColumns:
        - name: credit_card_due
          valueType: INT64
        - name: mortgage_due
          valueType: INT64
        - name: student_loan_due
          valueType: INT64
        - name: vehicle_loan_due
          valueType: INT64
        - name: hard_pulls
          valueType: INT64
        - name: missed_payments_2y
          valueType: INT64
        - name: missed_payments_1y
          valueType: INT64
        - name: missed_payments_6m
          valueType: INT64
        - name: bankruptcies
          valueType: INT64
        timestampField: created_timestamp
    